# CARMEN RVP Analysis notebook

In [1]:
# Imports

# General modules
import sys
import logging
import time
from os import path, makedirs
from gooey import Gooey, GooeyParser
import numpy as np

# import submodules
from parser import ArgParse
from analysis import ReadData, ItemsToList, LabelInputs
from plot import PlotHeatmap
from hitcalling import HitCallingFoldThreshold, CheckEC, CheckOutliers, CheckNDC, CheckDM, CheckEx, ConsiderControls

In [2]:
# Setup
outdir = "./test"
xname = "hehe"
layout = "./test/192_assignment.xlsx"
rawdata = "./1691225021.csv"
rawdataclean = "./test/1691225021-clean.csv"


# usually constant:
toi = 't12'
threshold = 1.8
alsort = 'original'
slsort = 'original'
ntcctrl = 'NTC'
cpcctrl = 'CPC'
ectrl = 'EC'
ndcctrl = 'NDC'
dmctrl = 'no-crRNA'
wctrl = 'water'
    
# Check that the output directory exists, and make it if not:
output_dir = path.expanduser(path.abspath(outdir))
if not path.exists(output_dir):
    try:
        makedirs(output_dir)
    except: 
        print('failed to create output directory {}'.format(output_dir))
if not path.isdir(output_dir): 
    print('specified output {} is not a directory'.format(output_dir))   

# Build the output file prefix:
output_prefix = path.join(outdir, xname)


# Check if everything is there
for inputfile in [layout, rawdata]:      
    if not path.exists(inputfile):
        error('specified file {} does not exist'.format(inputfile))      


In [3]:
# with open(rawdata, 'r') as inf, open(rawdataclean, 'w') as of:
#     for line in inf:
#         line = line.rstrip()
#         line = line.rstrip(',')
#         of.write(line)

In [4]:
# Read in data
probe_df, ref_df, bkgdprobe_df, bkgdref_df = ReadData(rawdata)

In [5]:
# Background subtraction
probe_df = probe_df - bkgdprobe_df
ref_df = ref_df - bkgdref_df

# Normalization with ROX
signal_df = probe_df/ref_df

In [6]:
import pandas as pd
from functools import reduce
def ItemsToList2(x, sheet, sort = 'original'):
    y = pd.read_excel(x, sheet_name = sheet)
    if sheet == 'assays':
        new_array = np.stack(y[['C1', 'C2', 'C3']].values, axis=-1)
    if sheet == 'samples':
        new_array = np.stack(y[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', \
                                'C7', 'C8', 'C9', 'C10', 'C11', 'C12', \
                                'C13', 'C14', 'C15', 'C16', 'C17', 'C18', \
                                'C19', 'C20', 'C21', 'C22', 'C23', 'C24']].values, axis=-1)
    itemlist =  np.concatenate(new_array).tolist()
    if sort == 'alphabetical':
        itemlist = np.unique(itemlist)
    if sort == 'original':
        #itemlist = [x[0] for x in groupby(itemlist)]
        itemlist = reduce(lambda l, x: l+[x] if x not in l else l, itemlist, [])
    
    return itemlist

In [7]:
# Assign sample and assay names
LabelInputs(signal_df, layout)

# Save signal dataframe to csv
signal_df.to_csv("{}_signal.csv".format(output_prefix))  
print("Normalized data saved to {}_signal.csv".format(output_prefix))

# Get median data for timepoint of interest
x_med = signal_df.groupby(['assay', 'sample']).median()[toi].unstack()
x_med.index.names=['']
x_med.columns.names=['']
median_df = x_med

Normalized data saved to ./test/hehe_signal.csv


In [8]:
# Create a list of all assays and all samples
a_list = ItemsToList(layout, 'assays', sort = alsort)
s_list = ItemsToList(layout, 'samples', sort = slsort)

In [10]:
a_list

['SARS-CoV-2',
 'HCoV-HKU1',
 'HCoV-NL63',
 'HCoV-OC43',
 'FLUAV',
 'FLUBV',
 'HMPV',
 'HRSV',
 'HPIV-3',
 'FLUAV-g4',
 'RNAseP',
 'no-crRNA']

In [11]:
signal_df.head()

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,sampleID,assayID,assay,sample
Chamber ID,,,,,,,,,,,,,,,,,
S012-A02,0.021956,0.021809,0.023976,0.023768,0.025479,0.024670,0.026526,0.025781,0.027195,0.025828,0.027725,0.026259,0.027974,S012,A02,HPIV-3,11_HKU1_1
S011-A02,0.021465,0.021653,0.023618,0.023542,0.025159,0.024653,0.026179,0.025079,0.026462,0.025717,0.027072,0.026311,0.027778,S011,A02,HPIV-3,9_HKU1_1
S010-A02,0.021115,0.021199,0.022767,0.022425,0.024020,0.023306,0.024933,0.024116,0.025719,0.024831,0.026199,0.025215,0.026761,S010,A02,HPIV-3,7_HKU1_1
S007-A02,0.021272,0.021538,0.024019,0.023889,0.025635,0.025256,0.026637,0.026140,0.027659,0.026479,0.028069,0.026886,0.028591,S007,A02,HPIV-3,1_HKU1_1
S008-A02,0.021627,0.021218,0.023512,0.023917,0.025500,0.025400,0.026822,0.025830,0.027311,0.026612,0.027713,0.026992,0.028291,S008,A02,HPIV-3,3_HKU1_1


In [12]:
def getCtrlMean(df,ctrl,cpcctrl):
    ctrl_values = df[ctrl].tolist()
    assaylist = df.index.values.tolist()
    
    if ctrl == cpcctrl:
        # Exclude no_crRNA control for CPC
        noCRNAidx = assaylist.index('no-crRNA')
        del ctrl_values[noCRNAidx]
    
    mean = np.mean(ctrl_values)
    std = np.std(ctrl_values)
    
    return(mean, std)

In [13]:
# Hit calling
neg_ctrl_mean, neg_ctrl_std = getCtrlMean(median_df,ntcctrl,cpcctrl)
pos_ctrl_mean, pos_ctrl_std = getCtrlMean(median_df,cpcctrl,cpcctrl)

# Overall dynamic range of the assay --> turns whole assay invalid
"""
The NTC and CPC are the ground truth for the hit calling. Since the validation of CPC and NTC depends on their group mean and standard deviation respectively, they have to be distinctively different from each other. For the assay to be valid, the ratio of separation band to dynamic range has to be greater than 0.2. The dynamic range is defined as the difference of the NTC mean and CPC mean. The separation band is the range between the CPC mean subtracted by three standard deviations of the CPCs and the NTC mean plus three standard deviations of the NTCs. 
"""
separation_band = (pos_ctrl_mean - 3*pos_ctrl_std) - (neg_ctrl_mean + 3* neg_ctrl_std)
dynamic_range = abs(pos_ctrl_mean - neg_ctrl_mean)
zfactor = separation_band/dynamic_range

if zfactor > 0.2:
    print('Z-Factor is {}'.format(zfactor))
else:
    print("Run invalid: the dynamic range of this assay is too low (Z-factor {}).".format(zfactor))
    print("CPC mean = {} stdev = {}; NTC mean = {} stdev = {}".format(pos_ctrl_mean, pos_ctrl_std,neg_ctrl_mean, neg_ctrl_std))
    PlotHeatmap(median_df, None, args.toi, a_list, s_list, output_prefix+'_LowDynamicRange')
    print("A heatmap has been generated nevertheless, but without hit calling.")

Z-Factor is 0.9380575837653264


In [14]:
# Check for RT-PCR control outlier --> turns assay invalid
NTC_outlier = CheckOutliers(median_df,ntcctrl,'negative')  
CPC_outlier, pass_cpc = CheckOutliers(median_df,cpcctrl,'positive')

# perform hit calling on remaining samples
hit_df, quanthit_df = HitCallingFoldThreshold(median_df,threshold,a_list,s_list,ntcctrl)
quanthit_df.T.to_csv("{}_HitQuantification.csv".format(output_prefix)) 
PlotHeatmap(median_df, hit_df, toi, a_list, s_list, output_prefix+'_initial_raw_hitcalling')

In [15]:
def CheckEC2(df,assaylist,ctrl):    
    outliers = []
    passed = True
    
    for guide in assaylist:
        test = df.loc[guide, ctrl]
        print(test)
        if guide == 'RNAseP':
            if df.loc[guide, ctrl] != 'positive':
                print('EC is not positive for RNAseP')
                print('Run is invalid because of failed extraction control.')
                passed = False
        else:
            if df.loc[guide, ctrl] != 'negative':
                print('EC is not negative for {}'.format(guide))
                outliers.append(guide)
        print("ok")
    
    print('EC outlier {}'.format(outliers))
    
    return outliers, passed

In [16]:
hit_df.loc['SARS-CoV-2','EC']

'negative'

In [18]:
# Extraction control should be negative for all targets and positive for RNAseP
EC_outlier, pass_ec = CheckEC(hit_df,a_list,ectrl)
print(EC_outlier)

[]


In [19]:
if pass_ec or pass_cpc == False:
    hit_df = hit_df.applymap(lambda x: 'invalid')
    hit_df.T.to_csv("{}_hits.csv".format(output_prefix))
    PlotHeatmap(median_df, hit_df, toi, a_list, s_list, output_prefix)
    print("Heatmap with hits is saved as {}_heatmap_{}.png".format(output_prefix, toi))

Heatmap with hits is saved as ./test/hehe_heatmap_t12.png


In [20]:
def CheckEx2(df,samplelist,assaylist):
    """ Sample-specific extraction control. 
    For all samples, at least one assay has to be positive otherwise this indicates a problem with extraction and the sample will be invalid. Repeat extraction, RT-PCR and crRNA-Cas13 testing for this specimen.
    """
    outliers = []
    dontcheck = [ectrl,ntcctrl,cpcctrl,dmctrl,'water','Water','W-Ext','w-ext','W-ext','W-det','w-det', wctrl] # exclude controls from this analysis
    for sample in samplelist:
        if sample in dontcheck:
            continue
        count1 = sum(1 for x in df[sample] if x == 'positive')
        if count1 == 0:
            outliers.append(sample)
            
    print('Extraction sample outlier {}'.format(outliers))
    return outliers

def ConsiderControls2(df,assaylist,samplelist,NTCout,CPCout,ECout,DSout,DMout,Exout):
    """ Replace called hit (1) or no hit (0) with invalid result (-1), or causing invalid result (-2) in hit dataframe.
    """
    resultx = 'invalid' # invalid
    resulty = 'invalid' # the sample causing the invalid result. For clinical reporting, both are named the same
    
    allguidesoutliers = NTCout + CPCout + ECout + DSout
    allsamplesoutliers = DMout + Exout
    for guide in assaylist:
        for sample in samplelist:
            # skip if guide/sample combination is fine
            if guide not in allguidesoutliers and sample not in allsamplesoutliers:
                continue
            # If negative RT-PCR control invalid, all other samples are invalid
            if guide in NTCout:
                if sample == ntcctrl:
                    df.loc[guide, sample] = resulty # this is causing the invalid result
                elif sample == cpcctrl and guide in CPCout:
                    df.loc[guide, sample] = resulty # this is also causing the invalid result
                elif sample == cpcctrl:
                    continue
                else:
                    df.loc[guide, sample] = resultx # all the others are invalid
            # If positive RT-PCR control invalid, all other samples, besides the negative control are invalid
            elif guide in CPCout:
                if sample == ntcctrl:
                    continue
                elif sample == cpcctrl:
                    df.loc[guide, sample] = resulty # this is causing the invalid result
                else:
                    df.loc[guide, sample] = resultx # all the others are invalid      
            # if any other control is invalid, all the samples besides controls are invalid
            else:
                if guide in ECout:
                    if sample == ectrl:
                        df.loc[guide, sample] = resulty # this is causing the invalid result
                    elif sample in [ntcctrl,cpcctrl,dmctrl,dmctrl]:
                        continue # this sample represents the ground truth and shouldn't be changed
                    else:
                        df.loc[guide, sample] = resultx # invalid
                if guide in DSout:
                    if sample == dmctrl:
                        df.loc[guide, sample] = resulty # this is causing the invalid result
                    elif sample in [ntcctrl,cpcctrl,ectrl,dmctrl,dmctrl]:
                        continue # this sample represents the ground truth and shouldn't be changed
                    else:
                        df.loc[guide, sample] = resultx # invalid
                if sample in DMout:
                    if guide == dmctrl:
                        df.loc[guide, sample] = resulty # this is causing the invalid result
                    else:
                        df.loc[guide, sample] = resultx # invalid
                if sample in Exout:
                    if guide == 'RNAseP':
                        df.loc[guide, sample] = resulty # this is causing the invalid result
                    elif guide == dmctrl:
                        continue
                    else:
                        df.loc[guide, sample] = resultx # invalid
    return df


In [21]:
# Negative Detection Control (NDC) for sample mastermix should be negative for all targets in W-det-NoMg
NDC_outlier = CheckNDC(hit_df,a_list,ndcctrl)
print(NDC_outlier)

# Detection control for assay mastermix should be negative for all sample in no-crRNA assay
DM_soutlier = CheckDM(hit_df,s_list,dmctrl) # this outlier list is with samples instead of assays
print("dn: ", DM_soutlier)

# All samples should be positive for RNAseP or at least another sample
Ex_soutlier = CheckEx2(hit_df,s_list,a_list) # check if extraction for the sample was successfull
print(Ex_soutlier)

# annotate samples that didn't pass the control
hit_df = ConsiderControls2(hit_df,a_list,s_list,NTC_outlier, CPC_outlier, EC_outlier, NDC_outlier, DM_soutlier, Ex_soutlier)

hit_df.T.to_csv("{}_hits.csv".format(output_prefix))            
logging.warning("Hit calling data frame is generated and saved to {}_hits.csv.".format(output_prefix))

# Plot heatmap with overlayed hits
PlotHeatmap(median_df, hit_df, toi, a_list, s_list, output_prefix)
logging.info("Heatmap with hits is generated and saved as {}_heatmap_{}.".format(output_prefix, toi))

['SARS-CoV-2', 'HCoV-HKU1', 'HCoV-NL63', 'HCoV-OC43', 'FLUAV', 'FLUBV', 'HMPV', 'HRSV', 'HPIV-3', 'FLUAV-g4', 'RNAseP', 'no-crRNA']
dn:  ['1_NL63_1', '1_NL63_0.5', '1_NL63_0.25', '1_NL63_0.125', '2_NL63_1', '2_NL63_0.5', '2_NL63_0.25', '2_NL63_0.125', '3_NL63_1', '3_NL63_0.5', '3_NL63_0.25', '3_NL63_0.125', '4_NL63_1', '4_NL63_0.5', '4_NL63_0.25', '4_NL63_0.125', '5_NL63_1', '5_NL63_0.5', '5_NL63_0.25', '5_NL63_0.125', '6_NL63_1', '6_NL63_0.5', '6_NL63_0.25', '6_NL63_0.125', '7_NL63_1', '7_NL63_0.5', '7_NL63_0.25', '7_NL63_0.125', '8_NL63_1', '8_NL63_0.5', '8_NL63_0.25', '8_NL63_0.125', '9_NL63_1', '9_NL63_0.5', '9_NL63_0.25', '9_NL63_0.125', '10_NL63_1', '10_NL63_0.5', '10_NL63_0.25', '10_NL63_0.125', '11_NL63_1', '11_NL63_0.5', '11_NL63_0.25', '11_NL63_0.125', '12_NL63_1', '12_NL63_0.5', '12_NL63_0.25', '12_NL63_0.125', '13_NL63_1', '13_NL63_0.5', '13_NL63_0.25', '13_NL63_0.125', '14_NL63_1', '14_NL63_0.5', '14_NL63_0.25', '14_NL63_0.125', '15_NL63_1', '15_NL63_0.5', '15_NL63_0.25', 

TypeError: can only concatenate tuple (not "list") to tuple